# LSTMs for Text Generation

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
%matplotlib inline
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
import tensorflow as tf
import csv

## Downloading Stories
Stories are automatically downloaded from https://www.cs.cmu.edu/~spok/grimmtmp/, if not detected in the disk. The total size of stories is around ~500KB. The dataset consists of 100 stories.

In [2]:
url = 'https://www.cs.cmu.edu/~spok/grimmtmp/'

# Create a directory if needed
dir_name = 'stories'
if not os.path.exists(dir_name):
    os.mkdir(dir_name)
    
def maybe_download(filename):
  """Download a file if not present"""
  print('Downloading file: ', dir_name+ os.sep+filename)
    
  if not os.path.exists(dir_name+os.sep+filename):
    filename, _ = urlretrieve(url + filename, dir_name+os.sep+filename)
  else:
    print('File ',filename, ' already exists.')
  
  return filename

num_files = 100
filenames = [format(i, '03d')+'.txt' for i in range(1,101)]

for fn in filenames:
    maybe_download(fn)

File  001.txt  already exists.
File  002.txt  already exists.
File  003.txt  already exists.
File  004.txt  already exists.
File  005.txt  already exists.
File  006.txt  already exists.
File  007.txt  already exists.
File  008.txt  already exists.
File  009.txt  already exists.
File  010.txt  already exists.
File  011.txt  already exists.
File  012.txt  already exists.
File  013.txt  already exists.
File  014.txt  already exists.
File  015.txt  already exists.
File  016.txt  already exists.
File  017.txt  already exists.
File  018.txt  already exists.
File  019.txt  already exists.
File  020.txt  already exists.
File  021.txt  already exists.
File  022.txt  already exists.
File  023.txt  already exists.
File  024.txt  already exists.
File  025.txt  already exists.
File  026.txt  already exists.
File  027.txt  already exists.
File  028.txt  already exists.
File  029.txt  already exists.
File  030.txt  already exists.
File  031.txt  already exists.
File  032.txt  already exists.
File  03

## Check if the files are downloaded. 
There should be 100 files.

In [3]:
for i in range(len(filenames)):
    file_exists = os.path.isfile(os.path.join(dir_name,filenames[i]))
    assert file_exists
print('%d files found.'%len(filenames))

100 files found.


## Reading data
Data will be stored in a list of lists where the each list represents a document and document is a list of words. We will then break the text into bigrams

In [4]:
def read_data(filename):
  
  with open(filename) as f:
    data = tf.compat.as_str(f.read())
    # make all the words lower case
    data = data.lower()
    data = list(data)
  return data

global documents
documents = []
for i in range(num_files):    
    print('\nProcessing file %s'%os.path.join(dir_name,filenames[i]))
    chars = read_data(os.path.join(dir_name,filenames[i]))
    
    # Break the data into bigrams
    two_grams = [''.join(chars[ch_i:ch_i+2]) for ch_i in range(0,len(chars)-2,2)]
    # Create a list of lists with bigrams
    documents.append(two_grams)
    print('Data size (Characters) (Document %d) %d' %(i,len(two_grams)))
    print('Sample string (Document %d) %s'%(i,two_grams[:50]))


Processing file stories\001.txt
Data size (Characters) (Document 0) 3666
Sample string (Document 0) ['in', ' o', 'ld', 'en', ' t', 'im', 'es', ' w', 'he', 'n ', 'wi', 'sh', 'in', 'g ', 'st', 'il', 'l ', 'he', 'lp', 'ed', ' o', 'ne', ', ', 'th', 'er', 'e ', 'li', 've', 'd ', 'a ', 'ki', 'ng', '\nw', 'ho', 'se', ' d', 'au', 'gh', 'te', 'rs', ' w', 'er', 'e ', 'al', 'l ', 'be', 'au', 'ti', 'fu', 'l,']

Processing file stories\002.txt
Data size (Characters) (Document 1) 4927
Sample string (Document 1) ['ha', 'rd', ' b', 'y ', 'a ', 'gr', 'ea', 't ', 'fo', 're', 'st', ' d', 'we', 'lt', ' a', ' w', 'oo', 'd-', 'cu', 'tt', 'er', ' w', 'it', 'h ', 'hi', 's ', 'wi', 'fe', ', ', 'wh', 'o ', 'ha', 'd ', 'an', '\no', 'nl', 'y ', 'ch', 'il', 'd,', ' a', ' l', 'it', 'tl', 'e ', 'gi', 'rl', ' t', 'hr', 'ee']

Processing file stories\003.txt
Data size (Characters) (Document 2) 9744
Sample string (Document 2) ['a ', 'ce', 'rt', 'ai', 'n ', 'fa', 'th', 'er', ' h', 'ad', ' t', 'wo', ' s', 'on', 's,', ' 

Data size (Characters) (Document 45) 7772
Sample string (Document 45) ['in', ' o', 'ld', 'en', ' t', 'im', 'es', ' t', 'he', 're', ' w', 'as', ' a', ' k', 'in', 'g,', ' w', 'ho', ' h', 'ad', ' b', 'eh', 'in', 'd ', 'hi', 's ', 'pa', 'la', 'ce', ' a', '\nb', 'ea', 'ut', 'if', 'ul', ' p', 'le', 'as', 'ur', 'e-', 'ga', 'rd', 'en', ' i', 'n ', 'wh', 'ic', 'h ', 'th', 'er']

Processing file stories\047.txt
Data size (Characters) (Document 46) 22157
Sample string (Document 46) ['th', 'er', 'e ', 'we', 're', ' o', 'nc', 'e ', 'up', 'on', ' a', ' t', 'im', 'e ', 'tw', 'o ', 'br', 'ot', 'he', 'rs', ', ', 'on', 'e ', 'ri', 'ch', ' a', 'nd', ' t', 'he', ' o', 'th', 'er', '\np', 'oo', 'r.', '  ', 'th', 'e ', 'ri', 'ch', ' o', 'ne', ' w', 'as', ' a', ' g', 'ol', 'ds', 'mi', 'th']

Processing file stories\048.txt
Data size (Characters) (Document 47) 2169
Sample string (Document 47) ['tw', 'o ', 'ki', 'ng', "s'", ' s', 'on', 's ', 'on', 'ce', ' w', 'en', 't ', 'ou', 't ', 'in', ' s', 'ea', 'rc', 'h '

Data size (Characters) (Document 98) 8258
Sample string (Document 98) ['**', '*t', 'he', 're', ' w', 'as', ' o', 'nc', 'e ', 'up', 'on', ' a', ' t', 'im', 'e ', 'a ', 'ki', 'ng', ' w', 'ho', ' h', 'ad', ' a', ' g', 're', 'at', ' f', 'or', 'es', 't ', 'ne', 'ar', '\nh', 'is', ' p', 'al', 'ac', 'e,', ' f', 'ul', 'l ', 'of', ' a', 'll', ' k', 'in', 'ds', ' o', 'f ', 'wi']

Processing file stories\100.txt
Data size (Characters) (Document 99) 2764
Sample string (Document 99) ['th', 'er', 'e ', 'wa', 's ', 'on', 'ce', ' a', ' w', 'on', 'de', 'rf', 'ul', ' m', 'us', 'ic', 'ia', 'n,', ' w', 'ho', ' w', 'en', 't ', 'qu', 'it', 'e ', 'fo', 'rl', 'or', 'n\n', 'th', 'ro', 'ug', 'h ', 'a ', 'fo', 're', 'st', ' a', 'nd', ' t', 'ho', 'ug', 'ht', ' o', 'f ', 'al', 'l ', 'ma', 'nn']


## Building the Dictionaries (Bigrams)
Builds the following. To understand each of these elements, let us also assume the text "I like to go to school"

* `dictionary`: maps a string word to an ID (e.g. {I:0, like:1, to:2, go:3, school:4})
* `reverse_dictionary`: maps an ID to a string word (e.g. {0:I, 1:like, 2:to, 3:go, 4:school}
* `count`: List of list of (word, frequency) elements (e.g. [(I,1),(like,1),(to,2),(go,1),(school,1)]
* `data` : Contain the string of text we read, where string words are replaced with word IDs (e.g. [0, 1, 2, 3, 2, 4])

It also introduces an additional special token `UNK` to denote rare words to are too rare to make use of.

In [5]:
def build_dataset(documents):
    chars = []
    # This is going to be a list of lists
    # Where the outer list denote each document
    # and the inner lists denote words in a given document
    data_list = []
  
    for d in documents:
        chars.extend(d)
    print('%d Characters found.'%len(chars))
    count = []
    # Get the bigram sorted by their frequency (Highest comes first)
    count.extend(collections.Counter(chars).most_common())
    
    # Create an ID for each bigram by giving the current length of the dictionary
    # And adding that item to the dictionary
    # Start with 'UNK' that is assigned to too rare words
    dictionary = dict({'UNK':0})
    for char, c in count:
        # Only add a bigram to dictionary if its frequency is more than 10
        if c > 10:
            dictionary[char] = len(dictionary)    
    
    unk_count = 0
    # Traverse through all the text we have
    # to replace each string word with the ID of the word
    for d in documents:
        data = list()
        for char in d:
            # If word is in the dictionary use the word ID,
            # else use the ID of the special token "UNK"
            if char in dictionary:
                index = dictionary[char]        
            else:
                index = dictionary['UNK']
                unk_count += 1
            data.append(index)
            
        data_list.append(data)
        
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
    return data_list, count, dictionary, reverse_dictionary

global data_list, count, dictionary, reverse_dictionary,vocabulary_size

# Print some statistics about data
data_list, count, dictionary, reverse_dictionary = build_dataset(documents)
print('Most common words (+UNK)', count[:5])
print('Least common words (+UNK)', count[-15:])
print('Sample data', data_list[0][:10])
print('Sample data', data_list[1][:10])
print('Vocabulary: ',len(dictionary))
vocabulary_size = len(dictionary)
del documents  # To reduce memory.

470747 Characters found.
Most common words (+UNK) [('e ', 15975), ('he', 15903), (' t', 14177), ('th', 13723), ('d ', 11175)]
Least common words (+UNK) [('c.', 1), ('"k', 1), ('pw', 1), ('f?', 1), (' z', 1), ('xq', 1), ('nm', 1), ('m?', 1), ('\t"', 1), ('\tw', 1), ('dj', 1), ('**', 1), ('*t', 1), ('ji', 1), ('tp', 1)]
Sample data [15, 28, 85, 23, 3, 95, 71, 11, 2, 16]
Sample data [22, 156, 25, 36, 83, 185, 43, 9, 89, 19]
Vocabulary:  544


## Generating Batches of Data
The following object generates a batch of data which will be used to train the LSTM. More specifically the generator breaks a given sequence of words into `batch_size` segments. We also maintain a cursor for each segment. So whenever we create a batch of data, we sample one item from each segment and update the cursor of each segment. 

In [6]:
class DataGeneratorOHE(object):
    
    def __init__(self,text,batch_size,num_unroll):
        # Text where a bigram is denoted by its ID
        self._text = text
        # Number of bigrams in the text
        self._text_size = len(self._text)
        # Number of datapoints in a batch of data
        self._batch_size = batch_size
        # Num unroll is the number of steps we unroll the RNN in a single training step
        # This relates to the truncated backpropagation we discuss in Chapter 6 text
        self._num_unroll = num_unroll
        # We break the text in to several segments and the batch of data is sampled by
        # sampling a single item from a single segment
        self._segments = self._text_size//self._batch_size
        self._cursor = [offset * self._segments for offset in range(self._batch_size)]
        
    def next_batch(self):
        '''
        Generates a single batch of data
        '''
        # Train inputs (one-hot-encoded) and train outputs (one-hot-encoded)
        batch_data = np.zeros((self._batch_size,vocabulary_size),dtype=np.float32)
        batch_labels = np.zeros((self._batch_size,vocabulary_size),dtype=np.float32)
        
        # Fill in the batch datapoint by datapoint
        for b in range(self._batch_size):
            # If the cursor of a given segment exceeds the segment length
            # we reset the cursor back to the beginning of that segment
            if self._cursor[b]+1>=self._text_size:
                self._cursor[b] = b * self._segments
            
            # Add the text at the cursor as the input
            batch_data[b,self._text[self._cursor[b]]] = 1.0
            # Add the preceding bigram as the label to be predicted
            batch_labels[b,self._text[self._cursor[b]+1]]= 1.0                       
            # Update the cursor
            self._cursor[b] = (self._cursor[b]+1)%self._text_size
                    
        return batch_data,batch_labels
        
    def unroll_batches(self):
        '''
        This produces a list of num_unroll batches
        as required by a single step of training of the RNN
        '''
        unroll_data,unroll_labels = [],[]
        for ui in range(self._num_unroll):
            data, labels = self.next_batch()            
            unroll_data.append(data)
            unroll_labels.append(labels)
        
        return unroll_data, unroll_labels
    
    def reset_indices(self):
        '''
        Used to reset all the cursors if needed
        '''
        self._cursor = [offset * self._segments for offset in range(self._batch_size)]
        
# Running a tiny set to see if things are correct
dg = DataGeneratorOHE(data_list[0][25:50],5,5)
print(data_list[0][25:50])
u_data, u_labels = dg.unroll_batches()

# Iterate through each data batch in the unrolled set of batches
for ui,(dat,lbl) in enumerate(zip(u_data,u_labels)):   
    print('\n\nUnrolled index %d'%ui)
    dat_ind = np.argmax(dat,axis=1)
    lbl_ind = np.argmax(lbl,axis=1)
    print('\tInputs:')
    for sing_dat in dat_ind:
        print('\t%s (%d)'%(reverse_dictionary[sing_dat],sing_dat),end=", ")
    print('\n\tOutput:')
    for sing_lbl in lbl_ind:        
        print('\t%s (%d)'%(reverse_dictionary[sing_lbl],sing_lbl),end=", ")

[1, 98, 42, 5, 83, 131, 33, 168, 63, 58, 48, 194, 106, 61, 136, 11, 14, 1, 84, 56, 72, 194, 111, 228, 259]


Unrolled index 0
	Inputs:
	e  (1), 	ki (131), 	 d (48), 	 w (11), 	be (72), 
	Output:
	li (98), 	ng (33), 	au (194), 	er (14), 	au (194), 

Unrolled index 1
	Inputs:
	li (98), 	ng (33), 	au (194), 	er (14), 	au (194), 
	Output:
	ve (42), 	
w (168), 	gh (106), 	e  (1), 	ti (111), 

Unrolled index 2
	Inputs:
	ve (42), 	
w (168), 	gh (106), 	e  (1), 	ti (111), 
	Output:
	d  (5), 	ho (63), 	te (61), 	al (84), 	fu (228), 

Unrolled index 3
	Inputs:
	d  (5), 	ho (63), 	te (61), 	al (84), 	fu (228), 
	Output:
	a  (83), 	se (58), 	rs (136), 	l  (56), 	l, (259), 

Unrolled index 4
	Inputs:
	a  (83), 	se (58), 	rs (136), 	l  (56), 	be (72), 
	Output:
	ki (131), 	 d (48), 	 w (11), 	be (72), 	au (194), 

## Defining the LSTM

This is a standard LSTM. The LSTM has 5 main components.
* Cell state
* Hidden state
* Input gate
* Forget gate
* Output gate

Each gate has three sets of weights (1 set for the current input, 1 set for the previous hidden state and 1 bias)

## Defining hyperparameters

Here we define several hyperparameters and are very similar to the ones we defined in Chapter 6. However additionally we use dropout; a technique that helps to avoid overfitting.

In [7]:
# Number of neurons in the hidden state variables
num_nodes = 128

# Number of data points in a batch we process
batch_size = 64

# Number of time steps we unroll for during optimization
num_unrollings = 50

dropout = 0.0 # We use dropout

# Use this in the CSV filename when saving
# when using dropout
filename_extension = ''
if dropout>0.0:
    filename_extension = '_dropout'
    
filename_to_save = 'lstm'+filename_extension+'.csv' # use to save perplexity values

## Defining Inputs and Outputs

In the code we define two different types of inputs. 
* Training inputs (The stories we downloaded) (batch_size > 1 with unrolling)
* Validation inputs (An unseen validation dataset) (bach_size =1, no unrolling)
* Test input (New story we are going to generate) (batch_size=1, no unrolling)

In [8]:
tf.reset_default_graph()

# Training Input data.
train_inputs, train_labels = [],[]

# Defining unrolled training inputs
for ui in range(num_unrollings):
    train_inputs.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size],name='train_inputs_%d'%ui))
    train_labels.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size], name = 'train_labels_%d'%ui))

# Validation data placeholders
valid_inputs = tf.placeholder(tf.float32, shape=[1,vocabulary_size],name='valid_inputs')
valid_labels = tf.placeholder(tf.float32, shape=[1,vocabulary_size], name = 'valid_labels')
# Text generation: batch 1, no unrolling.
test_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size], name = 'test_input')

## Defining Model Parameters

Now we define model parameters. Compared to RNNs, LSTMs have a large number of parameters. Each gate (input, forget, memory and output) has three different sets of parameters.

In [9]:
# Input gate (i_t) - How much memory to write to cell state
# Connects the current input to the input gate
ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], stddev=0.02))
# Connects the previous hidden state to the input gate
im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], stddev=0.02))
# Bias of the input gate
ib = tf.Variable(tf.random_uniform([1, num_nodes],-0.02, 0.02))

# Forget gate (f_t) - How much memory to discard from cell state
# Connects the current input to the forget gate
fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], stddev=0.02))
# Connects the previous hidden state to the forget gate
fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], stddev=0.02))
# Bias of the forget gate
fb = tf.Variable(tf.random_uniform([1, num_nodes],-0.02, 0.02))

# Candidate value (c~_t) - Used to compute the current cell state
# Connects the current input to the candidate
cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], stddev=0.02))
# Connects the previous hidden state to the candidate
cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], stddev=0.02))
# Bias of the candidate
cb = tf.Variable(tf.random_uniform([1, num_nodes],-0.02,0.02))

# Output gate (o_t) - How much memory to output from the cell state
# Connects the current input to the output gate
ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], stddev=0.02))
# Connects the previous hidden state to the output gate
om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], stddev=0.02))
# Bias of the output gate
ob = tf.Variable(tf.random_uniform([1, num_nodes],-0.02,0.02))


# Softmax Classifier weights and biases.
w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], stddev=0.02))
b = tf.Variable(tf.random_uniform([vocabulary_size],-0.02,0.02))

# Variables saving state across unrollings.
# Hidden state
saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False, name='train_hidden')
# Cell state
saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False, name='train_cell')

# Same variables for validation phase
saved_valid_output = tf.Variable(tf.zeros([1, num_nodes]),trainable=False, name='valid_hidden')
saved_valid_state = tf.Variable(tf.zeros([1, num_nodes]),trainable=False, name='valid_cell')

# Same variables for testing phase
saved_test_output = tf.Variable(tf.zeros([1, num_nodes]),trainable=False, name='test_hidden')
saved_test_state = tf.Variable(tf.zeros([1, num_nodes]),trainable=False, name='test_cell')

## Defining LSTM computations
Here first we define the LSTM cell computations as a consice function. Then we use this function to define training and test-time inference logic.

In [10]:
# Definition of the cell computation.
def lstm_cell(i, o, state):
    """Create an LSTM cell"""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state


In [11]:
# =========================================================
#Training related inference logic

# Keeps the calculated state outputs in all the unrollings
# Used to calculate loss
outputs = list()

# These two python variables are iteratively updated
# at each step of unrolling
output = saved_output
state = saved_state

# Compute the hidden state (output) and cell state (state)
# recursively for all the steps in unrolling
for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    output = tf.nn.dropout(output,keep_prob=1.0-dropout)
    # Append each computed output value
    outputs.append(output)

# calculate the score values
logits = tf.matmul(tf.concat(axis=0, values=outputs), w) + b
    
# Compute predictions.
train_prediction = tf.nn.softmax(logits)

# Compute training perplexity
train_perplexity_without_exp = tf.reduce_sum(tf.concat(train_labels,0)*-tf.log(tf.concat(train_prediction,0)+1e-10))/(num_unrollings*batch_size)

# ========================================================================
# Validation phase related inference logic

# Compute the LSTM cell output for validation data
valid_output, valid_state = lstm_cell(
    valid_inputs, saved_valid_output, saved_valid_state)
# Compute the logits
valid_logits = tf.nn.xw_plus_b(valid_output, w, b)

# Make sure that the state variables are updated
# before moving on to the next iteration of generation
with tf.control_dependencies([saved_valid_output.assign(valid_output),
                            saved_valid_state.assign(valid_state)]):
    valid_prediction = tf.nn.softmax(valid_logits)

# Compute validation perplexity
valid_perplexity_without_exp = tf.reduce_sum(valid_labels*-tf.log(valid_prediction+1e-10))

# ========================================================================
# Testing phase related inference logic

# Compute the LSTM cell output for testing data
test_output, test_state = lstm_cell(
test_input, saved_test_output, saved_test_state)

# Make sure that the state variables are updated
# before moving on to the next iteration of generation
with tf.control_dependencies([saved_test_output.assign(test_output),
                            saved_test_state.assign(test_state)]):
    test_prediction = tf.nn.softmax(tf.nn.xw_plus_b(test_output, w, b))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## Calculating LSTM Loss
We calculate the training loss of the LSTM here. It's a typical cross entropy loss calculated over all the scores we obtained for training data (`loss`).

In [12]:
# Before calcualting the training loss,
# save the hidden state and the cell state to
# their respective TensorFlow variables
with tf.control_dependencies([saved_output.assign(output),
                            saved_state.assign(state)]):

    # Calculate the training loss by
    # concatenating the results from all the unrolled time steps
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=logits, labels=tf.concat(axis=0, values=train_labels)))


## Defining Learning Rate and the Optimizer with Gradient Clipping
Here we define the learning rate and the optimizer we're going to use. We will be using the Adam optimizer as it is one of the best optimizers out there. Furthermore we use gradient clipping to prevent any gradient explosions.

In [13]:
# learning rate decay
gstep = tf.Variable(0,trainable=False,name='global_step')

# Running this operation will cause the value of gstep
# to increase, while in turn reducing the learning rate
inc_gstep = tf.assign(gstep, gstep+1)

# Decays learning rate everytime the gstep increases
tf_learning_rate = tf.train.exponential_decay(0.001,gstep,decay_steps=1, decay_rate=0.5)

# Adam Optimizer. And gradient clipping.
optimizer = tf.train.AdamOptimizer(tf_learning_rate)
gradients, v = zip(*optimizer.compute_gradients(loss))
# Clipping gradients
gradients, _ = tf.clip_by_global_norm(gradients, 5.0)

optimizer = optimizer.apply_gradients(
    zip(gradients, v))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


## Resetting Operations for Resetting Hidden States
Sometimes the state variable needs to be reset (e.g. when starting predictions at a beginning of a new epoch)

In [14]:
# Reset train state
reset_train_state = tf.group(tf.assign(saved_state, tf.zeros([batch_size, num_nodes])),
                          tf.assign(saved_output, tf.zeros([batch_size, num_nodes])))

# Reset valid state
reset_valid_state = tf.group(tf.assign(saved_valid_state, tf.zeros([1, num_nodes])),
                          tf.assign(saved_valid_output, tf.zeros([1, num_nodes])))

# Reset test state
reset_test_state = tf.group(
    saved_test_output.assign(tf.random_normal([1, num_nodes],stddev=0.05)),
    saved_test_state.assign(tf.random_normal([1, num_nodes],stddev=0.05)))


## Greedy Sampling to Break the Repetition
Here we write some simple logic to break the repetition in text. Specifically instead of always getting the word that gave this highest prediction probability, we sample randomly where the probability of being selected given by their prediction probabilities.

In [15]:
def sample(distribution):
  '''Greedy Sampling
  We pick the three best predictions given by the LSTM and sample
  one of them with very high probability of picking the best one'''

  best_inds = np.argsort(distribution)[-3:]
  best_probs = distribution[best_inds]/np.sum(distribution[best_inds])
  best_idx = np.random.choice(best_inds,p=best_probs)
  return best_idx

## Running the LSTM to Generate Text

Here we train the LSTM on the available data and generate text using the trained LSTM for several steps. From each document we extract text for `steps_per_document` steps to train the LSTM on. We also report the train perplexity at the end of each step. Finally we test the LSTM by asking it to generate some new text starting from a randomly picked bigram.

### Learning rate Decay Logic

Here we define the logic to decrease learning rate whenever the validation perplexity does not decrease

In [16]:
# Learning rate decay related
# If valid perpelxity does not decrease
# continuously for this many epochs
# decrease the learning rate
decay_threshold = 5
# Keep counting perplexity increases
decay_count = 0

min_perplexity = 1e10

# Learning rate decay logic
def decay_learning_rate(session, v_perplexity):
  global decay_threshold, decay_count, min_perplexity  
  # Decay learning rate
  if v_perplexity < min_perplexity:
    decay_count = 0
    min_perplexity= v_perplexity
  else:
    decay_count += 1

  if decay_count >= decay_threshold:
    print('\t Reducing learning rate')
    decay_count = 0
    session.run(inc_gstep)

### Running Training, Validation and Generation

We traing the LSTM on existing training data, check the validaiton perplexity on an unseen chunk of text and generate a fresh segment of text

In [ ]:
# Some hyperparameters needed for the training process

num_steps = 26
steps_per_document = 100
valid_summary = 1
train_doc_count = 100
docs_per_step = 10


# Capture the behavior of train perplexity over time
train_perplexity_ot = []
valid_perplexity_ot = []

session = tf.InteractiveSession()

# Initializing variables
tf.global_variables_initializer().run()
print('Initialized Global Variables ')

average_loss = 0 # Calculates the average loss ever few steps

# We use the first 10 documents that has 
# more than 10*steps_per_document bigrams for creating the validation dataset

# Identify the first 10 documents following the above condition
long_doc_ids = []
for di in range(num_files):
  if len(data_list[di])>10*steps_per_document:
    long_doc_ids.append(di)
  if len(long_doc_ids)==10:
    break
    
# Generating validation data
data_gens = []
valid_gens = []
for fi in range(num_files):
  # Get all the bigrams if the document id is not in the validation document ids
  if fi not in long_doc_ids:
    data_gens.append(DataGeneratorOHE(data_list[fi],batch_size,num_unrollings))
  # if the document is in the validation doc ids, only get up to the 
  # last steps_per_document bigrams and use the last steps_per_document bigrams as validation data
  else:
    data_gens.append(DataGeneratorOHE(data_list[fi][:-steps_per_document],batch_size,num_unrollings))
    # Defining the validation data generator
    valid_gens.append(DataGeneratorOHE(data_list[fi][-steps_per_document:],1,1))


feed_dict = {}
for step in range(num_steps):
    
    print('Training (Step: %d)'%step,end=' ')
    for di in np.random.permutation(train_doc_count)[:docs_per_step]:            
        doc_perplexity = 0
        for doc_step_id in range(steps_per_document):
            
            # Get a set of unrolled batches
            u_data, u_labels = data_gens[di].unroll_batches()
            
            # Populate the feed dict by using each of the data batches
            # present in the unrolled data
            for ui,(dat,lbl) in enumerate(zip(u_data,u_labels)):            
                feed_dict[train_inputs[ui]] = dat
                feed_dict[train_labels[ui]] = lbl
            
            # Running the TensorFlow operations
            _, l, step_perplexity = session.run([optimizer, loss, train_perplexity_without_exp], 
                                                       feed_dict=feed_dict)
            # Update doc_perpelxity variable
            doc_perplexity += step_perplexity 
            
            # Update the average_loss variable
            average_loss += step_perplexity 
            
        # Show the printing progress <train_doc_id_1>.<train_doc_id_2>. ...
        print('(%d).'%di,end='') 
    
        # resetting hidden state after processing a single document
        # It's still questionable if this adds value in terms of learning
        # One one hand it's intuitive to reset the state when learning a new document
        # On the other hand this approach creates a bias for the state to be zero
        # We encourage the reader to investigate further the effect of resetting the state
        #session.run(reset_train_state) # resetting hidden state for each document
    print('')
    
    # Generate new samples
    if (step+1) % valid_summary == 0:
      
      # Compute average loss
      average_loss = average_loss / (valid_summary*docs_per_step*steps_per_document)
      
      # Print losses
      print('Average loss at step %d: %f' % (step+1, average_loss))
      print('\tPerplexity at step %d: %f' %(step+1, np.exp(average_loss)))
      train_perplexity_ot.append(np.exp(average_loss))
        
      average_loss = 0 # reset loss
      valid_loss = 0 # reset loss
        
      # calculate valid perplexity
      for v_doc_id in range(10):
          # Remember we process things as bigrams
          # So need to divide by 2
          for v_step in range(steps_per_document//2):
            uvalid_data,uvalid_labels = valid_gens[v_doc_id].unroll_batches()        

            # Run validation phase related TensorFlow operations       
            v_perp = session.run(
                valid_perplexity_without_exp,
                feed_dict = {valid_inputs:uvalid_data[0],valid_labels: uvalid_labels[0]}
            )

            valid_loss += v_perp
            
          session.run(reset_valid_state)
      
          # Reset validation data generator cursor
          valid_gens[v_doc_id].reset_indices()      
    
      print()
      v_perplexity = np.exp(valid_loss/(steps_per_document*10.0//2))
      print("Valid Perplexity: %.2f\n"%v_perplexity)
      valid_perplexity_ot.append(v_perplexity)
          
      decay_learning_rate(session, v_perplexity)
        
      # Generating new text ...
      # We will be generating one segment having 500 bigrams
      # Feel free to generate several segments by changing
      # the value of segments_to_generate
      print('Generated Text after epoch %d ... '%step)  
      segments_to_generate = 1
      chars_in_segment = 500
    
      for _ in range(segments_to_generate):
        print('======================== New text Segment ==========================')
        
        # Start with a random word
        test_word = np.zeros((1,vocabulary_size),dtype=np.float32)
        rand_doc = data_list[np.random.randint(0,num_files)]
        test_word[0,rand_doc[np.random.randint(0,len(rand_doc))]] = 1.0
        print("\t",reverse_dictionary[np.argmax(test_word[0])],end='')
        
        # Generating words within a segment by feeding in the previous prediction
        # as the current input in a recursive manner
        for _ in range(chars_in_segment):    
          sample_pred = session.run(test_prediction, feed_dict = {test_input:test_word})  
          next_ind = sample(sample_pred.ravel())
          test_word = np.zeros((1,vocabulary_size),dtype=np.float32)
          test_word[0,next_ind] = 1.0
          print(reverse_dictionary[next_ind],end='')
        print("")
        
        # Reset train state
        session.run(reset_test_state)
        print('====================================================================')
      print("")

session.close()

# Write training and validation perplexities to a csv file
with open(filename_to_save, 'wt') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(train_perplexity_ot)
    writer.writerow(valid_perplexity_ot)

Initialized Global Variables 
Training (Step: 0) (63).(56).(72).(95).(28).(84).(9).(38).(13).(77).
Average loss at step 1: 4.380190
	Perplexity at step 1: 79.853177

Valid Perplexity: 61.75

Generated Text after epoch 0 ... 
======================== New text Segment ==========================
	 son, in you, and he haved withing ite to to the baun that had here in her and the had his was not her ould the peasant, and the haved the youth to thad hat hat had his was into that her ould the wast here in the
had hat here it he sould he haver you the haver the backing the hat hat here in the hat had his wall to him youth the will the peases he said, and sain, and sould to the ball sher as have her and he said ther have of he sed hat her the back the weat that the peasaid to his that him, but the
had the
had his will he senten the would the peases the you the mame, and his was into then the you and the
you said the wought hill than the hat the macking hered had her the you and the hat the had 

## LSTM with Beam-Search

Here we alter the previously defined prediction related TensorFlow operations to employ beam-search. Beam search is a way of predicting several time steps ahead. Concretely instead of predicting the best prediction we have at a given time step, we get predictions for several time steps and get the sequence of highest joint probability.

In [16]:
beam_length = 5 # number of steps to look ahead
beam_neighbors = 5 # number of neighbors to compare to at each step

# We redefine the sample generation with beam search
sample_beam_inputs = [tf.placeholder(tf.float32, shape=[1, vocabulary_size]) for _ in range(beam_neighbors)]

best_beam_index = tf.placeholder(shape=None, dtype=tf.int32)
best_neighbor_beam_indices = tf.placeholder(shape=[beam_neighbors], dtype=tf.int32)

# Maintains output of each beam
saved_sample_beam_output = [tf.Variable(tf.zeros([1, num_nodes])) for _ in range(beam_neighbors)]
# Maintains the state of each beam
saved_sample_beam_state = [tf.Variable(tf.zeros([1, num_nodes])) for _ in range(beam_neighbors)]

# Resetting the sample beam states (should be done at the beginning of each text snippet generation)
reset_sample_beam_state = tf.group(
    *[saved_sample_beam_output[vi].assign(tf.zeros([1, num_nodes])) for vi in range(beam_neighbors)],
    *[saved_sample_beam_state[vi].assign(tf.zeros([1, num_nodes])) for vi in range(beam_neighbors)]
)

# We stack them to perform gather operation below
stacked_beam_outputs = tf.stack(saved_sample_beam_output)
stacked_beam_states = tf.stack(saved_sample_beam_state)

# The beam states for each beam (there are beam_neighbor-many beams) needs to be updated at every depth of tree
# Consider an example where you have 3 classes where we get the best two neighbors (marked with star)
#     a`      b*       c  
#   / | \   / | \    / | \
#  a  b c  a* b` c  a  b  c
# Since both the candidates from level 2 comes from the parent b
# We need to update both states/outputs from saved_sample_beam_state/output to have index 1 (corresponding to parent b)
update_sample_beam_state = tf.group(
    *[saved_sample_beam_output[vi].assign(tf.gather_nd(stacked_beam_outputs,[best_neighbor_beam_indices[vi]])) for vi in range(beam_neighbors)],
    *[saved_sample_beam_state[vi].assign(tf.gather_nd(stacked_beam_states,[best_neighbor_beam_indices[vi]])) for vi in range(beam_neighbors)]
)

# We calculate lstm_cell state and output for each beam
sample_beam_outputs, sample_beam_states = [],[] 
for vi in range(beam_neighbors):
    tmp_output, tmp_state = lstm_cell(
        sample_beam_inputs[vi], saved_sample_beam_output[vi], saved_sample_beam_state[vi]
    )
    sample_beam_outputs.append(tmp_output)
    sample_beam_states.append(tmp_state)

# For a given set of beams, outputs a list of prediction vectors of size beam_neighbors
# each beam having the predictions for full vocabulary
sample_beam_predictions = []
for vi in range(beam_neighbors):
    with tf.control_dependencies([saved_sample_beam_output[vi].assign(sample_beam_outputs[vi]),
                                saved_sample_beam_state[vi].assign(sample_beam_states[vi])]):
        sample_beam_predictions.append(tf.nn.softmax(tf.nn.xw_plus_b(sample_beam_outputs[vi], w, b)))
        

## Running the LSTM with Beam Search to Generate Text

Here we train the LSTM on the available data and generate text using the trained LSTM for several steps. From each document we extract text for `steps_per_document` steps to train the LSTM on. We also report the train perplexity at the end of each step. Finally we test the LSTM by asking it to generate some new text with beam search starting from a randomly picked bigram.

### Learning rate Decay Logic

Here we define the logic to decrease learning rate whenever the validation perplexity does not decrease

In [17]:
# Learning rate decay related
# If valid perpelxity does not decrease
# continuously for this many epochs
# decrease the learning rate
decay_threshold = 5
# Keep counting perplexity increases
decay_count = 0

min_perplexity = 1e10

# Learning rate decay logic
def decay_learning_rate(session, v_perplexity):
  global decay_threshold, decay_count, min_perplexity  
  # Decay learning rate
  if v_perplexity < min_perplexity:
    decay_count = 0
    min_perplexity= v_perplexity
  else:
    decay_count += 1

  if decay_count >= decay_threshold:
    print('\t Reducing learning rate')
    decay_count = 0
    session.run(inc_gstep)

### Defining the Beam Prediction Logic
Here we define function that takes in the session as an argument and output a beam of predictions

In [18]:
test_word = None

def get_beam_prediction(session):
    
    # Generating words within a segment with Beam Search
    # To make some calculations clearer, we use the example as follows
    # We have three classes with beam_neighbors=2 (best candidate denoted by *, second best candidate denoted by `)
    # For simplicity we assume best candidate always have probability of 0.5 in output prediction
    # second best has 0.2 output prediction
    #           a`                   b*                   c                <--- root level
    #    /     |     \         /     |     \        /     |     \   
    #   a      b      c       a*     b`     c      a      b      c         <--- depth 1
    # / | \  / | \  / | \   / | \  / | \  / | \  / | \  / | \  / | \
    # a b c  a b c  a b c   a*b c  a`b c  a b c  a b c  a b c  a b c       <--- depth 2
    # So the best beams at depth 2 would be
    # b-a-a and b-b-a
        
    global test_word
    global sample_beam_predictions
    global update_sample_beam_state
    
    # Calculate the candidates at the root level
    feed_dict = {}
    for b_n_i in range(beam_neighbors):
        feed_dict.update({sample_beam_inputs[b_n_i]: test_word})

    # We calculate sample predictions for all neighbors with the same starting word/character
    # This is important to update the state for all instances of beam search
    sample_preds_root = session.run(sample_beam_predictions, feed_dict = feed_dict)  
    sample_preds_root = sample_preds_root[0]

    # indices of top-k candidates
    # b and a in our example (root level)
    this_level_candidates =  (np.argsort(sample_preds_root,axis=1).ravel()[::-1])[:beam_neighbors].tolist() 

    # probabilities of top-k candidates
    # 0.5 and 0.2
    this_level_probs = sample_preds_root[0,this_level_candidates] 

    # Update test sequence produced by each beam from the root level calculation
    # Test sequence looks like for our example (at root)
    # [b,a]
    test_sequences = ['' for _ in range(beam_neighbors)]
    for b_n_i in range(beam_neighbors):
        test_sequences[b_n_i] += reverse_dictionary[this_level_candidates[b_n_i]]

    # Make the calculations for the rest of the depth of the beam search tree
    for b_i in range(beam_length-1):
        test_words = [] # candidate words for each beam
        pred_words = [] # Predicted words of each beam

        # computing feed_dict for the beam search (except root)
        # feed dict should contain the best words/chars/bigrams found by the previous level of search

        # For level 1 in our example this would be
        # sample_beam_inputs[0]: b, sample_beam_inputs[1]:a
        feed_dict = {}
        for p_idx, pred_i in enumerate(this_level_candidates):                    
            # Updating the feed_dict for getting next predictions
            test_words.append(np.zeros((1,vocabulary_size),dtype=np.float32))
            test_words[p_idx][0,this_level_candidates[p_idx]] = 1.0

            feed_dict.update({sample_beam_inputs[p_idx]:test_words[p_idx]})

        # Calculating predictions for all neighbors in beams
        # This is a list of vectors where each vector is the prediction vector for a certain beam
        # For level 1 in our example, the prediction values for 
        #      b             a  (previous beam search results)
        # [a,  b,  c],  [a,  b,  c] (current level predictions) would be
        # [0.1,0.1,0.1],[0.5,0.2,0]
        sample_preds_all_neighbors = session.run(sample_beam_predictions, feed_dict=feed_dict)

        # Create a single vector with 
        # Making our example [0.1,0.1,0.1,0.5,0.2,0] 
        sample_preds_all_neighbors_concat = np.concatenate(sample_preds_all_neighbors,axis=1)

        # Update this_level_candidates to be used for the next iteration
        # And update the probabilities for each beam
        # In our example these would be [3,4] (indices with maximum value from above vector)
        this_level_candidates = np.argsort(sample_preds_all_neighbors_concat.ravel())[::-1][:beam_neighbors]

        # In the example this would be [1,1]
        parent_beam_indices = this_level_candidates//vocabulary_size

        # normalize this_level_candidates to fall between [0,vocabulary_size]
        # In this example this would be [0,1]
        this_level_candidates = (this_level_candidates%vocabulary_size).tolist()

        # Here we update the final state of each beam to be
        # the state that was at the index 1. Because for both the candidates at this level the parent is 
        # at index 1 (that is b from root level)
        session.run(update_sample_beam_state, feed_dict={best_neighbor_beam_indices: parent_beam_indices})

        # Here we update the joint probabilities of each beam and add the newly found candidates to the sequence
        tmp_this_level_probs = np.asarray(this_level_probs) #This is currently [0.5,0.2]
        tmp_test_sequences = list(test_sequences) # This is currently [b,a]

        for b_n_i in range(beam_neighbors):
            # We make the b_n_i element of this_level_probs to be the probability of parents
            # In the example the parent indices are [1,1]
            # So this_level_probs become [0.5,0.5]
            this_level_probs[b_n_i] = tmp_this_level_probs[parent_beam_indices[b_n_i]]

            # Next we multipyle these by the probabilities of the best candidates from current level 
            # [0.5*0.5, 0.5*0.2] = [0.25,0.1]
            this_level_probs[b_n_i] *= sample_preds_all_neighbors[parent_beam_indices[b_n_i]][0,this_level_candidates[b_n_i]]

            # Make the b_n_i element of test_sequences to be the correct parent of the current best candidates
            # In the example this becomes [b, b]
            test_sequences[b_n_i] = tmp_test_sequences[parent_beam_indices[b_n_i]]

            # Now we append the current best candidates
            # In this example this becomes [ba,bb]
            test_sequences[b_n_i] += reverse_dictionary[this_level_candidates[b_n_i]]

            # Create one-hot-encoded representation for each candidate
            pred_words.append(np.zeros((1,vocabulary_size),dtype=np.float32))
            pred_words[b_n_i][0,this_level_candidates[b_n_i]] = 1.0

    # Calculate best beam id based on the highest beam probability
    # Using the highest beam probability always lead to very monotonic text
    # Let us sample one randomly where one being sampled is decided by the likelihood of that beam
    rand_cand_ids = np.argsort(this_level_probs)[-3:]
    rand_cand_probs = this_level_probs[rand_cand_ids]/np.sum(this_level_probs[rand_cand_ids])
    random_id = np.random.choice(rand_cand_ids,p=rand_cand_probs)

    best_beam_id = parent_beam_indices[random_id]

    # Update state and output variables for test prediction
    session.run(update_sample_beam_state,feed_dict={best_neighbor_beam_indices:[best_beam_id for _ in range(beam_neighbors)]})

    # Make the last word/character/bigram from the best beam
    test_word = pred_words[best_beam_id]
    
    return test_sequences[best_beam_id]



NameError: name 'test_sequences' is not defined

### Running Training, Validation and Generation

We traing the LSTM on existing training data, check the validaiton perplexity on an unseen chunk of text and generate a fresh segment of text

In [21]:
filename_to_save = 'lstm_beam_search_dropout'

# Some hyperparameters needed for the training process

num_steps = 26
steps_per_document = 100
valid_summary = 1
train_doc_count = 100
docs_per_step = 10


beam_nodes = []

beam_train_perplexity_ot = []
beam_valid_perplexity_ot = []
session = tf.InteractiveSession()

tf.global_variables_initializer().run()

print('Initialized')
average_loss = 0

# We use the first 10 documents that has 
# more than 10*steps_per_document bigrams for creating the validation dataset

# Identify the first 10 documents following the above condition
long_doc_ids = []
for di in range(num_files):
  if len(data_list[di])>10*steps_per_document:
    long_doc_ids.append(di)
  if len(long_doc_ids)==10:
    break
    
# Generating validation data
data_gens = []
valid_gens = []
for fi in range(num_files):
  # Get all the bigrams if the document id is not in the validation document ids
  if fi not in long_doc_ids:
    data_gens.append(DataGeneratorOHE(data_list[fi],batch_size,num_unrollings))
  # if the document is in the validation doc ids, only get up to the 
  # last steps_per_document bigrams and use the last steps_per_document bigrams as validation data
  else:
    data_gens.append(DataGeneratorOHE(data_list[fi][:-steps_per_document],batch_size,num_unrollings))
    # Defining the validation data generator
    valid_gens.append(DataGeneratorOHE(data_list[fi][-steps_per_document:],1,1))


feed_dict = {}
for step in range(num_steps):
    
    for di in np.random.permutation(train_doc_count)[:docs_per_step]:            
        doc_perplexity = 0
        for doc_step_id in range(steps_per_document):
            
            # Get a set of unrolled batches
            u_data, u_labels = data_gens[di].unroll_batches()
            
            # Populate the feed dict by using each of the data batches
            # present in the unrolled data
            for ui,(dat,lbl) in enumerate(zip(u_data,u_labels)):            
                feed_dict[train_inputs[ui]] = dat
                feed_dict[train_labels[ui]] = lbl
            
            # Running the TensorFlow operations
            _, l, step_perplexity = session.run([optimizer, loss, train_perplexity_without_exp], 
                                                       feed_dict=feed_dict)
            # Update doc_perpelxity variable
            doc_perplexity += step_perplexity 
            
            # Update the average_loss variable
            average_loss += step_perplexity 
            
        # Show the printing progress <train_doc_id_1>.<train_doc_id_2>. ...
        print('(%d).'%di,end='') 
    
    # resetting hidden state after processing a single document
    # It's still questionable if this adds value in terms of learning
    # One one hand it's intuitive to reset the state when learning a new document
    # On the other hand this approach creates a bias for the state to be zero
    # We encourage the reader to investigate further the effect of resetting the state
    #session.run(reset_train_state) # resetting hidden state for each document
    print('')
    
    if (step+1) % valid_summary == 0:
      
      # Compute average loss
      average_loss = average_loss / (docs_per_step*steps_per_document*valid_summary)
      
      # Print loss
      print('Average loss at step %d: %f' % (step+1, average_loss))
      print('\tPerplexity at step %d: %f' %(step+1, np.exp(average_loss)))
      beam_train_perplexity_ot.append(np.exp(average_loss))
    
      average_loss = 0 # reset loss
        
      valid_loss = 0 # reset loss
        
      # calculate valid perplexity
      for v_doc_id in range(10):
          # Remember we process things as bigrams
          # So need to divide by 2
          for v_step in range(steps_per_document//2):
            uvalid_data,uvalid_labels = valid_gens[v_doc_id].unroll_batches()        

            # Run validation phase related TensorFlow operations       
            v_perp = session.run(
                valid_perplexity_without_exp,
                feed_dict = {valid_inputs:uvalid_data[0],valid_labels: uvalid_labels[0]}
            )

            valid_loss += v_perp
            
          session.run(reset_valid_state)
      
          # Reset validation data generator cursor
          valid_gens[v_doc_id].reset_indices()      
    
      print()
      v_perplexity = np.exp(valid_loss/(steps_per_document*10.0//2))
      print("Valid Perplexity: %.2f\n"%v_perplexity)
      beam_valid_perplexity_ot.append(v_perplexity)
      
      # Decay learning rate
      decay_learning_rate(session, v_perplexity)
    
      # Generating new text ...
      # We will be generating one segment having 500 bigrams
      # Feel free to generate several segments by changing
      # the value of segments_to_generate
      print('Generated Text after epoch %d ... '%step)  
      segments_to_generate = 1
      chars_in_segment = 500//beam_length
    
      for _ in range(segments_to_generate):
        print('======================== New text Segment ==========================')
        # first word randomly generated
        test_word = np.zeros((1,vocabulary_size),dtype=np.float32)
        rand_doc = data_list[np.random.randint(0,num_files)]
        test_word[0,rand_doc[np.random.randint(0,len(rand_doc))]] = 1.0
        print("",reverse_dictionary[np.argmax(test_word[0])],end='')
        
        for _ in range(chars_in_segment):
            
            test_sequence = get_beam_prediction(session)
            print(test_sequence,end='')
            
        print("")
        session.run([reset_sample_beam_state])
        
        print('====================================================================')
      print("")

session.close()
    
with open(filename_to_save, 'wt') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(beam_train_perplexity_ot)
    writer.writerow(beam_valid_perplexity_ot)

Initialized
(40).(71).(52).(81).(84).(8).(13).(19).(63).(28).
Average loss at step 1: 4.428126
	Perplexity at step 1: 83.774273

Valid Perplexity: 74.72

Generated Text after epoch 0 ... 
======================== New text Segment ==========================
 .  with the laught to laught and in the wang the chas the was laugh, and but ther hang the changeling to to her hang ther hang, and took to they her, and, anythe she was that he was to to her was the wang, who
hen her was to to the was the wang to laugh, and but her, who
child, she her, who
child, she begand he her, whicen to the to the cames, and he her, who
child, and her, when her, when her, whic her, whice, and in laugh, and with the would, anyegan the begand her was the wang, when ther hat they her, and, anythe to the cames, and he her, who
child, and her, when her, when her, whic her, whice, and in laugh, and but ther hang the cegand her was the wang, who
hen the was took her, but the wang to her was the wang, when ther hangel

KeyboardInterrupt: 